## Required Libraries

In [1]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for mapping json data
import lib.handle_json as hjson
# library for connecting to the db
import lib.handle_db as dbh

# datetime parsing
from datetime import datetime
# clear the output after each loop cycle
from IPython.display import clear_output

## Global variables

In [2]:
db_conn = dbh.DataBaseAdapter('./db_files/production.sqlite3')
#db_conn = dbh.DataBaseAdapter('../mcc_data/development.sqlite3')

# global lists of affiliation components
#institutions_list = []
#countries_list = []
#school_list = []
#department_list = []
#faculty_list = []
#group_list = []

#def refresh_lists():
db_conn = dbh.DataBaseAdapter('../mcc_data/development.sqlite3')
# get institutions list from affiliations table
print("Refreshing lists")
institutions_list = db_conn.get_value_list("affiliations", "institution")
# get coutries from affiliations table
countries_list = db_conn.get_value_list("affiliations","country")
# get school list from affiliations table
school_list = db_conn.get_value_list("Affiliations","school")
# get department list from affiliations table
department_list = db_conn.get_value_list("affiliations","department")
# get faculty list from affiliations table
faculty_list = db_conn.get_value_list("affiliations","faculty")
# get research group list from affiliations table
group_list = db_conn.get_value_list("affiliations", "work_group")

def refresh_lists(affi_values):
    if affi_values['institution'] != "" and not affi_values['institution'] in institutions_list:
        institutions_list.append (affi_values['institution'])
    if affi_values['department'] != "" and not affi_values['department'] in department_list:
        department_list.append(affi_values['department'])
    if affi_values['faculty'] != "" and not affi_values['faculty'] in faculty_list:
        faculty_list.append(affi_values['faculty'])
    if affi_values['work_group'] != "" and not affi_values['work_group'] in group_list:
        group_list.append(affi_values['work_group'])
    if affi_values['school'] != "" and not affi_values['school'] in  school_list:
        school_list.append( affi_values['school'])
    if affi_values['country'] != "" and not affi_values['country'] in countries_list:
        countries_list.append(affi_values['country'])
    
affi_keys = {'a':'institution', 'b':'country', 'c':'department','d':'faculty',
             'e':'work_group'}

country_synonyms = {"(UK)":"United Kingdom", "UK":"United Kingdom",
                    "U.K.":"United Kingdom", "U. K.":"United Kingdom",
                    "U.K":"United Kingdom", "PRC":"Peoples Republic of China",
                    "P.R.C.":"Peoples Republic of China", 
                    "P.R.China":"Peoples Republic of China",
                    "P. R. China":"Peoples Republic of China",
                    "P.R. China":"Peoples Republic of China","China":"Peoples Republic of China",
                    "United States":"United States of America",
                    "USA":"United States of America","U.S.A.":"United States of America",
                    "U. S. A.":"United States of America", "U.S.":"United States of America",
                    "U. S.":"United States of America","US":"United States of America",
                   }

institution_synonyms = {"Paul Scherrer Institut":"Paul Scherrer Institute",
                        "PSI":"Paul Scherrer Institute",
                        "Diamond Light source Ltd": "Diamond Light Source Ltd.",
                        "Diamond Light Source": "Diamond Light Source Ltd.",
                        "University of St Andrews": "University of St. Andrews",
                        "Queen’s University of Belfast":"Queen's University Belfast",
                        "STFC":"Science and Technology Facilities Council",
                        "University of Manchester": "The University of Manchester",
                        "Finden Limited": "Finden Ltd",
                        "The ISIS facility":"ISIS Neutron and Muon Source",
                        "ISIS Neutron and Muon Facility":"ISIS Neutron and Muon Source",
                        "ISIS Pulsed Neutron and Muon Facility":"ISIS Neutron and Muon Source",
                        "STFC":"Science and Technology Facilities Council",
                        "Oxford University":"University of Oxford",
                        "University of St Andrews":"University of St. Andrews",
                        "Diamond Light Source Ltd Harwell Science and Innovation Campus":"Diamond Light Source Ltd.",
                        "Diamond Light Source":"Diamond Light Source Ltd.",
                        "ISIS Facility":"ISIS Neutron and Muon Source",
                        "University College of London":"University College London",
                        "UCL":"University College London", "UOP LLC":"UOP LLC, A Honeywell Company",
                        "University of Manchester":"The University of Manchester",
                        "Johnson-Matthey Technology Centre":"Johnson Matthey Technology Centre",
                        "Research Complex at Harwell (RCaH)":"Research Complex at Harwell",
                        "RCaH":"Research Complex at Harwell",
                        "Queens University Belfast":"Queen's University Belfast",
                        "Queen’s University Belfast":"Queen's University Belfast",
                        "University of Edinburgh":"The University of Edinburgh",
                        "SynCat@Beijing, Synfuels China Technology Co. Ltd.":"SynCat@Beijing Synfuels China Company Limited",
                        "Synfuels China Compnay Limited":"SynCat@Beijing Synfuels China Company Limited",
                        "Finden Limited":"Finden Ltd",
                        "The UK Catalysis Hub":"UK Catalysis Hub",
                        "Univ. Pablo de Olavide":"Universidad Pablo de Olavide",
                        "Univ Rennes":"Université de Rennes",
                        "Université Rennes":"Université de Rennes",
                        "Institut Laue Langevin":"Institut Laue-Langevin",
                        "Esfera UAB":"Universitat Autònoma de Barcelona",
                        "Kings College London":"King's College London",
                       }  

country_exceptions = ["Denmark Hill", "UK Catalysis Hub", "Sasol Technology U.K.", "N. Ireland", 'Indian']



Refreshing lists


## Custom Processing Functions

In [3]:
# process addresses which are a single string
def process_single_affi(cf_affi_id, art_aut_id, cr_affi_str):
    clear_output()
    print('*********************************************************')
    print("* Art. author", art_aut_id, "Affi:", cr_affi_str, "Affi ID:", cr_affi_id)
    # 1. split the single string using keywords list and synonyms tables
    affi_values = split_single(cr_affi_str)
    print("* Affi Values:", affi_values)
    affiliation_id = address_id = 0
    # determine if parsing needs help (institution found?)   
    if affi_values['institution'] in ["", None]:
        affi_values = custom_split(affi_values)
    if not affi_values['institution'] in ["", None]:
        affiliation_id, address_id = affiliation_exists(affi_values)
        if affiliation_id !=0 and address_id != 0:
            print("* Affiliation found in DB id:", affiliation_id, "Address:", address_id)
            print('* Add ')
        elif affiliation_id !=0 and address_id == 0:
            print("* Affiliation found in DB id:", affiliation_id, "Need to add address:", affi_values['address'])
            address_id = add_address(affi_values['address'], affiliation_id, affi_values['country'])
            print("* Added address:", affi_values['address'], "ID:", address_id)
        else:
            print("* Need to add affiliation and address")
            print ('* Try to add affi for', affi_values)
            address_string = affi_values['address']
            affiliation_id = add_affiliation(affi_values)
            # renew lists as new affi is added
            refresh_lists(affi_values)
            address_id = add_address(address_string, affiliation_id, affi_values['country'])
            print ('* Added affiliation and address', affiliation_id, address_id)
            
        if affiliation_id !=0 and address_id != 0:
            print("* Adding author affiliation record ")
            added_aut_affi_id = add_author_affiliation(art_aut_id, affiliation_id, address_id)
            print("* Added author affiliation record ID:", added_aut_affi_id)
            update_cr_affiliation(cf_affi_id, added_aut_affi_id)
            print("* Update cr affiliation", cf_affi_id)
    else: 
        print('could not process string:', cr_affi_str)
        print('results obtained:', affi_values)
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

# Check the if any of the values in the list is in the given string
def check_list(a_string, a_list):
    return_str = ""
    temp_str = ""
    for an_item in a_list:
        if an_item in a_string:
            if len(an_item) > len(temp_str):
                temp_str = an_item
    if len(temp_str)>0:
        a_string = a_string.replace(temp_str,"")
        return_str = temp_str
    return return_str, a_string

# verify if the string has some of the synomyms in the provided synonym table
def str_has_synonym(affi_str, synonym_dict):
    ret_str = ""
    temp_str = ""
    for a_key in synonym_dict.keys():
        if a_key in affi_str:
            if len(a_key) > len(temp_str):
                temp_str = a_key
    if len(temp_str) > 0:
        ret_str = synonym_dict[temp_str]
        affi_str = affi_str.replace(temp_str,'')
    return ret_str, affi_str

def custom_split(affiliation_values):
    # Look for institution name should be in the address element
    print ("Select how to parse ",  affiliation_values)
    option_how = 0
    while not option_how in [1,2,3,4]:
        print('options: ')
        print('1: get from address element')
        print('2: enter manually')
        print('3: manual parse')
        print('4: do nothing ')
        print('selection:')
        str_opt = input()
        if str_opt != '':
            option_how = int(str_opt)
        else:
            option_how = 4
    if option_how == 1:
        affiliation_values = get_from_address(affiliation_values, 'institution')
    elif option_how == 2:
        affiliation_values = institution_manual_entry(affiliation_values)
    elif option_how == 3:
        affiliation_values = manual_split_address(affiliation_values)
    return affiliation_values

def manual_split_address(affiliation_values):
    opt_end = False
    while (opt_end == False):
        print(affiliation_values)
        missing_vals = {}
        int_idx = 1
        for an_element in affiliation_values:
            if affiliation_values[an_element] == "":
                missing_vals[int_idx] = an_element
                int_idx += 1
        print('Which element to get from string:')
        for an_idx in missing_vals:
            print(" ", an_idx, missing_vals[an_idx])
        print (" ", len(missing_vals)+1, 'end')
        print('Selection')
        option_sel = int(input())
        if option_sel == len(missing_vals)+1:
            opt_end = True
        elif option_sel > 0 and option_sel <= len(missing_vals):
            print("Try to extract ", missing_vals[option_sel])
            affiliation_values[missing_vals[option_sel]], affiliation_values['address'] = \
            get_from_string(affiliation_values['address'], missing_vals[option_sel])
    return affiliation_values
        
def get_from_string(string_value, element):
    element_value = ""
    print("************ get",element,"*************************")
    print("split and add this: ", string_value)
    element_value = extract_custom(string_value)
    string_value = string_value.replace(element_value,'')
    string_value = remove_extra_commas(string_value)
    return element_value, string_value

def get_from_address(affiliation_values, element): 
    affiliation_values[element], affiliation_values['address'] = \
      get_from_string(affiliation_values['address'], element)
    return affiliation_values

def institution_manual_entry(affiliation_values):
    print("*********** enter institution ************************")
    print("Provide institution for:", affiliation_values)            
    ins_str = input()
    affiliation_values['institution'] = ins_str
    return affiliation_values

def extract_custom(split_this):
    print(split_this)
    decimal_str = ""
    for indx in range(0, len(split_this)):
        val = " "
        if (indx % 10 == 0): val = str(int(indx/10)) 
        decimal_str += val
    print(decimal_str)
    unit_str = ""
    for indx in range(0, len(split_this)):
        val = str(indx%10)
        if (indx % 10 == 0): val = "0" 
        unit_str += val
    print(unit_str)
    print("start")
    str_start = int(input())
    print("end")
    str_end = int(input())
    return split_this[str_start:str_end+1]
    
def remove_extra_commas(str_this):
    str_this = str_this.replace(", ,", "")
    str_this = str_this.replace("; ;", "")
    str_this = str_this.replace(" ;", ";")
    str_this = str_this.strip()
    if len(str_this) == 0:
        return ""
    if str_this[0] in (",",";") : str_this = str_this[1:] 
    if str_this[-1:] in (",",";") : str_this = str_this[:-1] 
    return str_this.strip()

def split_single(affiliation_str):
    inst_str = dept_str = faculty_str = group_str = ctry_str = school_str = ""

    splitting_this = affiliation_str
    # lookup using institution synonyms table
    inst_str, splitting_this = str_has_synonym(splitting_this, institution_synonyms)
    # Lookup using list of institutions 
    if inst_str == "":
        inst_str, splitting_this = check_list(splitting_this,institutions_list)

    #  lookup using Country Synonyms table
    ctry_str, splitting_this = str_has_synonym(splitting_this, country_synonyms)
    #  lookup using Countries list        
    if ctry_str == "":
        ctry_str, splitting_this = check_list(splitting_this, countries_list)

    # Lookup using department list
    dept_str, splitting_this = check_list(splitting_this, department_list)

    # Lookup using school list
    school_str, splitting_this = check_list(splitting_this, school_list)

    # Lookup using faculty list
    faculty_str, splitting_this = check_list(splitting_this, faculty_list)
    
    # Lookup using group list
    group_str, splitting_this = check_list(splitting_this, group_list)
    
    splitting_this = remove_extra_commas(splitting_this)
    
    return {'institution':inst_str, 'school': school_str, 
            'department': dept_str, 'faculty': faculty_str, 
            'work_group': group_str, 'country': ctry_str, 'address':  splitting_this}

def get_affi_address(affiliation_id):
    return_val = 0
    add_id = db_conn.get_value("addresses", "id", "affiliation_id", affiliation_id)
    if  add_id == None or add_id == []:
        return_val = 0
    else:
        return_val = add_id[0]
    return return_val

def make_author_affiliation(art_aut_id, affi_values, addr_values):
    # get smallest unit
    smallest_unit = "" 
    #id Institution> Faculty > School > Department > Work_group + address + Country
    if affi_values[4] != None and  len(affi_values[4]) > 0: #'work_group'
        smallest_unit = affi_values[4]
    elif affi_values[2] != None and len(affi_values[2]) > 0 and smallest_unit == "": #'department'
        smallest_unit = affi_values[2]
    elif affi_values[9] != None and  len(affi_values[9]) > 0 and smallest_unit == "": #'school'
        smallest_unit = affi_values[9]
    elif affi_values[3] != None and len(affi_values[3]) > 0 and smallest_unit == "": #'faculty'
        smallest_unit = affi_values[3]
       
    ret_parsed = {}
    ret_parsed['article_author_id'] = art_aut_id
    if len(smallest_unit) > 0:
        ret_parsed['name'] = smallest_unit + ", "+  affi_values[1] #'institution'
    else:
        ret_parsed['name'] = affi_values[1]
    ret_parsed['short_name'] = affi_values[1]
    add_01 = ""
    if affi_values[3] != None and affi_values[3]  != "" and affi_values[3] != smallest_unit:
        add_01 = affi_values[3] 
    if affi_values[9] != None and affi_values[9] != "" and affi_values[9] != smallest_unit:
        if add_01 != "":
               add_01 += ", "+ affi_values[9]
        else:
               add_01 += affi_values[9]
    if affi_values[2] != None and affi_values[2] != "" and affi_values[2] != smallest_unit:
        if add_01 != "":
               add_01 += ", "+ affi_values[2]
        else:
               add_01 += affi_values[2]
    if add_01 != "":
        ret_parsed['add_01'] = add_01
        ret_parsed['add_02'] = addr_values[1] 
        ret_parsed['add_03'] = addr_values[2]
        ret_parsed['add_04'] = addr_values[3]
        ret_parsed['add_05'] = addr_values[4]
    else:
        ret_parsed['add_01'] = addr_values[1]
        ret_parsed['add_02'] = addr_values[2] 
        ret_parsed['add_03'] = addr_values[3]
        ret_parsed['add_04'] = addr_values[4]
        
    ret_parsed['country'] = addr_values[5]
    ret_parsed['affiliation_id'] = affi_values[0]
    ret_parsed['created_at'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    ret_parsed['updated_at'] = ret_parsed['created_at'] 
    return ret_parsed


def add_author_affiliation(art_aut_id, affi_id, add_id):
    print("Arguments",art_aut_id, affi_id, add_id)
    affiliation_row = list(db_conn.get_row("affiliations", affi_id))[0]
    address_row = list(db_conn.get_row("addresses", add_id))[0]
    print("Affiliation values", affiliation_row)
    print("Address values", address_row )
    new_aut_affi = make_author_affiliation(art_aut_id, affiliation_row, address_row)
    new_aa_id = db_conn.put_values_table("author_affiliations", new_aut_affi.keys(), new_aut_affi.values())
    return new_aa_id

def add_address(addr_str, affi_id, ctry_str):
    add_update_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    affi_address = {'add_01':addr_str,"affiliation_id":affi_id,'country':ctry_str,
                    'created_at': add_update_time, 'updated_at':add_update_time}
    add_id = db_conn.put_values_table("addresses", affi_address.keys(), affi_address.values())
    return add_id

def add_affiliation(affi_values):
    add_update_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    affiliation_new = affi_values
    del affiliation_new['address']
    if 'address' in affiliation_new.keys():
        del affiliation_new['address']
    if 'num' in affiliation_new.keys():
        del affiliation_new['num']
    affiliation_new['created_at'] = add_update_time
    affiliation_new['updated_at'] = add_update_time
    affiliation_id = db_conn.put_values_table("affiliations", affiliation_new.keys(), affiliation_new.values())
    return affiliation_id
    
def affiliation_exists(affi_values):
    qry_where = ""
    affi_id = add_id = 0
    if affi_values['institution'] != "":
        qry_where += "institution = \"" + affi_values['institution'] + "\""
    if affi_values['department'] != "":
        qry_where += " AND department = '" + affi_values['department'] + "'"
    if affi_values['faculty'] != "":
        qry_where += " AND faculty = '" + affi_values['faculty'] + "'"
    if affi_values['work_group'] != "":
        qry_where += " AND work_group = '" + affi_values['work_group'] + "'"
    if affi_values['school'] != "":
        qry_where += " AND school = '" + affi_values['school'] + "'"
    if affi_values['country'] != "":
        qry_where += " AND country = '" + affi_values['country'] + "'"
    
    if (qry_where[0:4] == " AND"):
        qry_where= qry_where[4:]
    print("Query:", qry_where)    
    if qry_where == "":
        return 0,0
    
    result = db_conn.get_values("affiliations","id",qry_where)
    if result != []:
        affi_id = result[0][0]
        add_id = get_affi_address(affi_id)
    return affi_id, add_id

def update_cr_affiliation(cr_affi_id, aut_affi_id):
    db_conn.set_value_table('cr_affiliations', cr_affi_id, 'author_affiliation_id', aut_affi_id)


def get_flat_list(comp_list):
    flat_list = []
    for sublist in comp_list:
            for item in sublist:
                flat_list.append(item)
    return flat_list

def aut_ids_unmapped_cr_affiliations():
    aut_affi_ids = db_conn.get_values('cr_affiliations', 'article_author_id',
                                      'NOT author_affiliation_id is NULL GROUP BY article_author_id')# HAVING count() = 1')
    return get_flat_list(aut_affi_ids)

# process affiliations in single strings >>

In [4]:
#refresh_lists()
# this part runs OK on single affi strings
aut_affi_ids = db_conn.get_values('cr_affiliations', 'article_author_id',
                                  'author_affiliation_id is NULL')
flat_list = []
for sublist in aut_affi_ids:
    for item in sublist:
        flat_list.append(item)

i_idx = 0
aut_affi_ids = get_flat_list(aut_affi_ids)
for aut_afi_id in aut_affi_ids:
    non_parsed_affis = db_conn.get_values('cr_affiliations', 'name', 'article_author_id = '+ str(aut_afi_id) +\
                                          ' and author_affiliation_id is NULL')
    non_parsed_cr_affi_ids = db_conn.get_values('cr_affiliations', 'id', 'article_author_id = '+ str(aut_afi_id)+\
                                          ' and author_affiliation_id is NULL')
    if len(non_parsed_affis) == 1:
        cr_affi_id = non_parsed_cr_affi_ids[0][0]
        process_single_affi(cr_affi_id, aut_afi_id, non_parsed_affis[0][0].replace('\r',' '))
   

In [5]:
# add manually parsed affis >>

In [7]:
# add manually parsed affis
csv_affis = '../mcc_data/new_affis.csv'
affi_data, cols= csvh.get_csv_data(csv_affis, 'num')
for an_affi  in affi_data:
    print (affi_data[an_affi])
    affiliation_id, address_id = affiliation_exists(affi_data[an_affi])
    if affiliation_id !=0 and address_id != 0:
        print("* Affiliation found in DB id:", affiliation_id, "Address:", address_id)
        print('* Add ')
    elif affiliation_id !=0 and address_id == 0:
        print("* Affiliation found in DB id:", affiliation_id, "Need to add address:", affi_data[an_affi]['address'])
        address_id = add_address(affi_data[an_affi]['address'], affiliation_id, affi_data[an_affi]['country'])
        print("* Added address:", affi_data[an_affi]['address'], "ID:", address_id)
    else:
        print("* Need to add affiliation and address")
        print ('* Try to add affi for', affi_data[an_affi])
        address_string = affi_data[an_affi]['address']
        affiliation_id = add_affiliation(affi_data[an_affi])
        # renew lists as new affi is added
        #refresh_lists(affi_values)
        address_id = add_address(address_string, affiliation_id, affi_data[an_affi]['country'])
        #print ('* Added affiliation and address', affiliation_id, address_id)
    

{'num': '1', 'institution': 'Advanced Light Source', 'department': '', 'faculty': '', 'work_group': '', 'country': 'United States of America', 'sector': 'Research Facility', 'school': '', 'address': 'Lawrence Berkeley National Laboratory, Berkeley, California 94720'}
Query: institution = "Advanced Light Source" AND country = 'United States of America'
* Affiliation found in DB id: 564 Address: 191
* Add 
{'num': '2', 'institution': 'Aix-Marseille Université CNRS', 'department': '', 'faculty': '', 'work_group': '', 'country': 'France', 'sector': 'Academia', 'school': '', 'address': 'IM2NP UMR 7334, 13397 Marseille'}
Query: institution = "Aix-Marseille Université CNRS" AND country = 'France'
* Affiliation found in DB id: 565 Address: 192
* Add 
{'num': '3', 'institution': 'ALISTORE European Research Institute', 'department': '', 'faculty': '', 'work_group': '', 'country': 'France', 'sector': 'Research Facility', 'school': '', 'address': 'FR CNRS 3104, 80039 Cedex Amiens'}
Query: institut

# UNDER DEVELOPMENT CODE >>>

In [8]:
# return a flat list from a complex list
def flatten_list(comp_list):
    flat_list = []
    for sublist in comp_list:
            for item in sublist:
                flat_list.append(item)
    return flat_list

# get lits of article_authors with unmapped crossref affiliations
def aut_ids_unmapped_cr_affiliations():
    aut_affi_ids = db_conn.get_values('cr_affiliations', 'article_author_id',
                                      'author_affiliation_id is NULL GROUP BY article_author_id')# HAVING count() = 1')
    return flatten_list(aut_affi_ids)

def cr_affiliation_ids_for_author(aut_affi_id):
    non_parsed_cr_affi_ids = db_conn.get_values('cr_affiliations', 'id', 'article_author_id = '+ str(aut_affi_id) +\
                                          ' and author_affiliation_id is NULL')
    return flatten_list(non_parsed_cr_affi_ids)

def get_cr_values(cr_ids):
    filter_str = str(tuple(cr_ids))
    if (filter_str[-2:-1]) == ",":
        filter_str = filter_str[:-2]+")"
    cr_values = db_conn.get_values('cr_affiliations', 'id,name', 'id in' + filter_str)
    return cr_values

def is_string_affi(affi_string):
    #affi string shoud include the institution else it is not an affi string
    affi_values = split_single(affi_string)
    non_empty_count = 0
    if affi_values['institution'] == "":
        return False
    for an_element in affi_values:
        if affi_values[an_element] != "" and an_element != 'address':
            non_empty_count += 1
    if non_empty_count > 1:
        print(affi_values)
        return True
    else:
        return False

def remove_blanks(affi_values):
    remove_keys = []
    for a_key in affi_values:
        if affi_values[a_key] == "":
            remove_keys.append(a_key)
    for a_key in remove_keys:
        del affi_values[a_key]
    return affi_values

def get_affi_vals(a_str):
    ret_vals = split_single(a_str.replace('\r',' '))
    # omit address vals too
    del ret_vals['address']
    return remove_blanks(ret_vals)
    
def one_affi(affi_hash):
    valid_keys = ["institution","work_group","department","faculty","country","school"]
    find_str = ""
    for item_key in affi_hash.keys():
        if affi_hash[item_key] != "" and item_key in valid_keys:
            find_str += item_key + " = '" + affi_hash[item_key].replace("'","''") + "' and "
    if find_str != "":
        find_str = find_str[:-5]
        print(find_str)
        found = db_conn.get_values('affiliations', 'id', find_str)
        if len(found) > 0 :
            return True
    return False
    
def manual_map_complex(affiliation_values, complex_vals):
    opt_end = False
    while (opt_end == False):
        print ("*** Manual Mapping ***")
        print("* Parsed:   ", affiliation_values)
        print("* Original: ", complex_vals)
        missing_vals = {}
        int_idx = 1
        for an_element in affiliation_values:
            if affiliation_values[an_element] == "":
                missing_vals[int_idx] = an_element
                int_idx += 1
        print('Which element to get from string:')
        for an_idx in missing_vals:
            print(" ", an_idx, missing_vals[an_idx])
        print (" ", len(missing_vals)+1, 'end')
        print('Selection')
        option_sel = int(input())
        if option_sel == len(missing_vals)+1:
            opt_end = True
        elif option_sel > 0 and option_sel <= len(missing_vals):
            print("Map: ", missing_vals[option_sel])
            affiliation_values[missing_vals[option_sel]] = \
                get_from_list(complex_vals, missing_vals[option_sel])
    return affiliation_values

def get_from_list(complex_vals, element):
    element_value = ""
    print("************ get",element,"*************************")
    print("split or map this: ", complex_vals)
    option_how = 0
    while not option_how in [1,2,3]:
        print('options: ')
        print('1: map complex element')
        print('2: enter manually')
        print('3: do nothing ')
        print('selection:')
        try:
            option_how = int(input())
        except:
            option_how = 0
    if option_how == 1:
        element_value = map_complex(complex_vals, element)
    elif option_how == 2:
        print('enter value')
        element_value = input()  
    return element_value

def map_complex(complex_vals, element):
    return_value = ""
    print('mapping')
    opt_end = False
    while (opt_end == False):
        print ("*** Manual Mapping ",element,"***")
        missing_vals = {}
        int_idx = 1
        for an_element in complex_vals:
            missing_vals[int_idx] = complex_vals[an_element]
            int_idx += 1
        print('Which value corresponds to ',element+':')
        for an_idx in missing_vals:
            print(" ", an_idx, missing_vals[an_idx])
        print (" ", len(missing_vals)+1, 'end')
        print('Selection')
        option_sel = int(input())
        if option_sel > 0 and option_sel <= len(missing_vals):
            return_value = missing_vals[option_sel]
            opt_end = True
        elif option_sel == len(missing_vals) + 1:
            opt_end = True
    return return_value

In [ ]:
def merge_affi_vals(affi_val_1, affi_val_2):
    ret_affi_vals = affi_val_1
    for an_element in affi_val_2:
        if affi_val_2[an_element] != '': #and an_element != 'address':
            affi_val_1[an_element] = affi_val_2[an_element]           
    return affi_val_1

def process_complex_affi(complex_affi, art_aut_id):
    print ("process as complex affiliation in strings:", complex_affi )
    current_affi = {}
    test_affi={}
    temp_affi={}
    same_affi = True # one institution is found
    for cr_id in complex_affi:
        temp_split = split_single(complex_affi[cr_id])
        if temp_split!={}:
            if test_affi == {}:
                test_affi = temp_split
            else:
                temp_affi = merge_affi_vals(test_affi, temp_split)
                print("temp affi:", temp_affi)
                affi_id, add_id = affiliation_exists(temp_affi)
                if affi_id != 0 and temp_affi['institution']!="": # some affiliation exists, consistency
                    if current_affi == {}:
                        current_affi = temp_affi
                    elif (temp_affi['institution'] == current_affi['institution']):
                        current_affi = temp_affi
                    else:
                        same_affi = False
                        print ("* Institution changed", temp_affi)
                        print("save current")
    
    if 'institution' in temp_affi.keys() and temp_affi['institution']!="": # affi does not exist
        print ("new affi:", temp_affi,"\nAdd it?") 
        opt_sel = 0
        while (not opt_sel in [1,2]):
            print("1. Add" )
            print("2. Ingore" )
            try:
                opt_sel = int(input())
            except:
                opt_sel = 0
        if opt_sel == 1:
            current_affi = temp_affi

    if 'institution' in temp_affi.keys() and temp_affi['institution']=="":
        print ("not parsed correctLY? ", temp_affi, complex_affi)
        current_affi = manual_map_complex(temp_affi, complex_affi)
        
    if current_affi != {}:
        print("save current", current_affi)
        affiliation_id, address_id = affiliation_exists(current_affi)
        if affiliation_id !=0 and address_id != 0:
            print("* Affiliation found in DB id:", affiliation_id, "Address:", address_id)
            print('* Add ')
        elif affiliation_id !=0 and address_id == 0:
            print("* Affiliation found in DB id:", affiliation_id, "Need to add address:", current_affi['address'])
            address_id = add_address(current_affi['address'], affiliation_id, current_affi['country'])
            print("* Added address:", current_affi['address'], "ID:", address_id)
        else:
            print("* Need to add affiliation and address")
            print ('* Try to add affi for', current_affi)
            address_string = current_affi['address']
            affiliation_id = add_affiliation(current_affi)
            
            address_id = add_address(address_string, affiliation_id, current_affi['country'])
            print ('* Added affiliation and address', affiliation_id, address_id)
            
        if affiliation_id !=0 and address_id != 0:
            print("* Adding author affiliation record ")
            added_aut_affi_id = add_author_affiliation(art_aut_id, affiliation_id, address_id)
            print("* Added author affiliation record ID:", added_aut_affi_id)
            for cr_id in complex_affi:
                update_cr_affiliation(cr_id, added_aut_affi_id)
                print("* Update cr affiliation", cr_id)
        
# get lits of article author ids with unmapped crossref affiliations
art_aut_ids = aut_ids_unmapped_cr_affiliations()
for art_aut_id in art_aut_ids:
    print ('Art. author ID:', art_aut_id)
    # get cr ids for art. author
    non_parsed_cr_affi_ids = cr_affiliation_ids_for_author(art_aut_id)     
    cr_values = get_cr_values(non_parsed_cr_affi_ids)
    #print ("CrossRef values ", cr_values)
    complex_affi = {}
    for a_cr_val in cr_values:
        if is_string_affi(a_cr_val[1]): 
            print("process as single affiliation in string: ", a_cr_val[0], a_cr_val[1],"\n author:", art_aut_id)
            #process_single_affi(a_cr_val[0], aut_affi_id, a_cr_val[1])
            affi_values = split_single(a_cr_val[1])
            #print("processed as", affi_values)
        else:
            print("element: ", a_cr_val[1])
            complex_affi[a_cr_val[0]] = a_cr_val[1].replace('\r',' ')
    if len(complex_affi)>0:
        process_complex_affi(complex_affi, art_aut_id)
            
            

In [9]:
one_affi({'institution': 'East China University of Science and Technology', 'line_1': 'Shanghai 200237'})

institution = 'East China University of Science and Technology'


True

# Incomplete code >>>

In [ ]:
def merge_affi_vals(affi_val_1, affi_val_2):
    ret_affi_vals = affi_val_1
    for an_element in affi_val_2:
        if affi_val_2[an_element] != '' and an_element != 'address':
            affi_val_1[an_element] = affi_val_2[an_element]   
        # merge all unmatched strings into address
        #if affi_val_2['address'] != "" and affi_val_1['address'] != "":
        #    affi_val_1['address'] += ", "+ affi_val_2['address']
        #elif affi_val_2['address'] != "":
        #    affi_val_1['address'] = affi_val_2['address']
            
    return affi_val_1

def process_complex_affi(complex_affi, aut_affi_id):
    print ("process as complex affiliation in strings:", complex_affi )
    # take the first element
    # add it to affi_hash if affi_lines empty
    # in next element try to see if the string belongs to same affiliation
    # after first line try adding to affi_hash, if resulting in same affi, add, else start new affi_hash item
    #   - test if current affiliation is full (i.e. inst, dep, sch, fcty, wg, cty.)
    #     or already contains the elements just returned
    #     - not complete then test if it fits with current affiliation
    #       - if an affi is element already filled then mark current affi as
    #         - complete and create a new affi.
    #           - mark as complete requires looking for matches and missing minimal parts in affiliations table: i.e. inst, ctry,
    #           - if inst and ctry blank, use the same as last one (i.e. inst and ctry from previous)
    # currently there is no affiliation

    affi_hash = {}
    hash_indx = 0
    affi_hash[hash_indx] = None
    current_affi = {}
    test_affi={}
    for cr_id in complex_affi:
        temp_split = split_single(complex_affi[cr_id])
        if temp_split!={}:
            if test_affi == {}:
                test_affi = temp_split
            else:
                temp_affi = merge_affi_vals(test_affi, temp_split) 
                affi_id, add_id = affiliation_exists(temp_affi)
                if affi_id != 0:
                    print ("affi exists")
    print(test_affi)
       
# get lits of article_authors with unmapped crossref affiliations
aut_affi_ids = aut_ids_unmapped_cr_affiliations()
for aut_affi_id in aut_affi_ids:
    if aut_affi_id>2089:
        print ('Art. author ID:', aut_affi_id)
        # get cr ids for art. author
        non_parsed_cr_affi_ids = cr_affiliation_ids_for_author(aut_affi_id)     
        #print(aut_affi_id)
        #print(non_parsed_affis)
        #print(non_parsed_cr_affi_ids)
        # get values for cr_ids
        cr_values = get_cr_values(non_parsed_cr_affi_ids)
        #print ("CrossRef values ", cr_values)
        complex_affi = {}
        other_lines = 1
        affi_hash = {}
        hash_indx = 0
        affi_hash[hash_indx] = None
        for a_cr_val in cr_values:
            temp_split = get_affi_vals(a_cr_val[1])
            if temp_split == {}:
                temp_split["line_"+str(other_lines)] = a_cr_val[1]
                other_lines += 1
            print("TEMP split", temp_split)
            add_another = False      
            if affi_hash[hash_indx] == None:
                # if all elements are from same affi just add them
                if one_affi(temp_split):
                    affi_hash[hash_indx] = [temp_split,[a_cr_val[0]]]
            # add element if it belongs to same affi
            elif affi_hash[hash_indx] != None:
                current_pair = affi_hash[hash_indx]
                print("Current pair",current_pair)
                if set(current_pair[0].keys()).isdisjoint(set(temp_split.keys())):
                    temp_pair = [merge_affi_vals(current_pair[0], temp_split),current_pair[1].append(a_cr_val[0])]
                    print("TEMP PAIR", temp_pair, current_pair[1], a_cr_val[0])
                    if one_affi(temp_pair[0]):
                        affi_hash[hash_indx] = [temp_split,[a_cr_val[0]]]
                    else :
                        add_another = True
            if add_another:
                hash_indx+=1
                affi_hash[hash_indx] = [temp_split,[a_cr_val[0]]]
                add_another = False
        ####################################################        
            print (affi_hash)
            
            #if temps_split
            if is_string_affi(a_cr_val[1]): 
                print("process as single affiliation in string: ", a_cr_val[0], a_cr_val[1],"\n author:", aut_affi_id)
                #process_single_affi(a_cr_val[0], aut_affi_id, a_cr_val[1])
                affi_values = split_single(a_cr_val[1])
                #print("processed as", affi_values)
            else:
                print("element: ", a_cr_val[1])
                complex_affi[a_cr_val[0]] = a_cr_val[1].replace('\r',' ')
        if len(complex_affi)>0:
            process_complex_affi(complex_affi, aut_affi_id)
    #if aut_affi_id>150:
    #    break
        

In [ ]:
non_parsed_affi = "Some string\rwith a return"
print (non_parsed_affi)
non_parsed_affi.replace('\r',' ')
print (non_parsed_affi.replace('\r',' '))
affi_vals = {'institution': '', 'school': 'Chemical Engineering', 'department': '', 'faculty': '', 'work_group': '', 'country': '', 'address': 'UNILAB, State Key Lab of , School of'}
print(affi_vals)
affi_vals = remove_blanks(affi_vals)
print(affi_vals)

In [ ]:
# old version


# Split the affiliation
# if affiliatin is found in DB:
#    retrieve affiliation ID and address ID
# else:
#    add affiliation and address to the DB
#    get the IDs of affiliation and address
# return affiliation ID and address ID
def db_split(affiliation):
    affiliation_id = 0
    address_id = 0
    fields={'a':'institution', 'b':'country', 'c':'department','d':'faculty',
            'e':'work_group','f':'address', 'g':'school'}
    print (affiliation, len(affiliation))
    for indx in range(0, len(affiliation)):
        inst_str = dept_str = faculty_str = group_str = ctry_str = school_str = ""
        qry_where_str = ""
        addr_list=[]
        
        checking_this = affiliation[indx]['name']
        #print (affiliation[indx]['name'])
        for inst in institution_synonyms.keys():
            if inst in checking_this:
                inst_str = institution_synonyms[inst]
                checking_this = checking_this.replace(inst,'')
                break
        # Institution 
        if inst_str == "":
            inst_str, checking_this = check_list(checking_this,institutions_list)

        # Country Synonyms
        for ctry in country_synonyms.keys():
            if ctry in checking_this:
                ctry_str = country_synonyms[ctry]
                checking_this = checking_this.replace(ctry,'')
                break
        
        for ctry in countries_list:
            if ctry in checking_this:
                ctry_str = ctry
                checking_this = checking_this.replace(ctry,'')
                break
        
        # see if department in affiliation string
        dept_str, checking_this = check_list(checking_this, department_list)
            
        # find schools in affiliation string
        school_str, checking_this = check_list(checking_this, school_list)

                
        for fclty in faculty_list:
            if fclty in checking_this:
                faculty_str = fclty
                checking_this = checking_this.replace(fclty,'')
                break
        for grp in group_list:
            if grp in checking_this:
                group_str = grp
                checking_this = checking_this.replace(grp,'')
                break
        checking_this = checking_this.strip()
        checking_this = checking_this.replace("  ", " ")
        addr_list.append(checking_this)
        
        result=-1
        print ('Institution:', inst_str)
        print ('School:', school_str)
        print ('Department:', dept_str)
        print ('Faculty:', faculty_str)
        print ('Group:', group_str)
        print ('Country:', ctry_str)

        print ('Address',  addr_list)
        if inst_str != "":
            affiliation_id, address_id = find_or_add(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list)
        else:        
            print("***************** get institution **************************************")
            print("split and add this: ", checking_this)
            addr_list.pop(0)
            ins_str = extract_custom(checking_this)
            checking_this = checking_this.replace(ins_str,'')
            checking_this = checking_this.strip()
            checking_this = checking_this.replace("  ", " ")
            addr_list.append(checking_this)
            affiliation_id, address_id = find_or_add(ins_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list)
    return affiliation_id, address_id

def add_affi_to_db(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list, qry_where):
    ret_parsed = {}
    ret_parsed['institution'] = inst_str
    ret_parsed['school'] = school_str
    ret_parsed['department'] = dept_str
    ret_parsed['faculty'] = faculty_str
    ret_parsed['work_group'] = group_str
    ret_parsed['country'] = ctry_str
    ret_parsed['created_at'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    ret_parsed['updated_at'] = ret_parsed['created_at'] 
    print("** Will Add:", ret_parsed, " with address ", addr_list)
    db_conn.put_values_table("affiliations", ret_parsed.keys(), ret_parsed.values())
    result = db_conn.get_values("affiliations","id",qry_where)
    affi_id = result[0][0]
    print("Added:", result, len(result))
    addr_str = "; ".join(addr_list)
    affi_address = {'add_01':addr_str,"affiliation_id":affi_id,'country':ctry_str,
                    'created_at':ret_parsed['created_at'], 'updated_at':ret_parsed['created_at']}
    db_conn.put_values_table("addresses", affi_address.keys(), affi_address.values())
    add_id = db_conn.get_value("addresses", "id", "affiliation_id", affi_id)[0]
    print('affiliation id:', affi_id, " address id: ", add_id)
    return affi_id, add_id

def build_aut_affi(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list, art_aut_id, affi_id, addr_str):
    # get smallest unit
    smallest_unit = "" 
    #id Institution> Faculty > School > Department > Work_group + address + Country
    if len(group_str) > 0:
        smallest_unit = group_str
    elif len(dept_str) > 0 and smallest_unit == "":
        smallest_unit = dept_str
    elif len(school_str) > 0 and smallest_unit == "":
        smallest_unit = school_str
    elif len(faculty_str) > 0 and smallest_unit == "":
        smallest_unit = faculty_str
       
    ret_parsed = {}
    ret_parsed['article_author_id'] = art_aut_id
    if len(smallest_unit) > 0:
        ret_parsed['name'] = smallest_unit + ", "+  inst_str
        ret_parsed['short_name'] = inst_str
    else:
        ret_parsed['name'] = inst_str
        ret_parsed['short_name'] = inst_str
    add_01 = ""
    if faculty_str != "" and faculty_str != smallest_unit:
        add_01 = faculty_str
    if school_str != "" and school_str != smallest_unit:
        if add_01 != "":
               add_01 += ", "+ school_str
        else:
               add_01 += school_str
    if dept_str != "" and dept_str != smallest_unit:
        if add_01 != "":
               add_01 += ", "+ dept_str
        else:
               add_01 += dept_str
    ret_parsed['add_01'] = add_01
    ret_parsed['add_02'] = addr_str
    ret_parsed['add_03'] = ctry_str
    ret_parsed['country'] = ctry_str
    ret_parsed['affiliation_id'] = affi_id
    ret_parsed['created_at'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    ret_parsed['updated_at'] = ret_parsed['created_at'] 
    return ret_parsed
                   

# add a new author affi record
def add_author_affi_to_db(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list, art_aut_id, affi_id, addr_str):  
    # build record for author affiliation    
    ret_parsed = build_aut_affi(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list, art_aut_id, affi_id, addr_str)
    print("** Will Add:", ret_parsed)
    aut_affi_id = db_conn.put_values_table("author_affiliations", ret_parsed.keys(), ret_parsed.values())
    return aut_affi_id;

def find_or_add(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list):
    qry_where = ""
    if inst_str != "":
        qry_where += "institution = '" + inst_str + "'"
        if dept_str != "":
            qry_where += " AND department = '" + dept_str + "'"
        if faculty_str != "":
            qry_where += " AND faculty = '" + faculty_str + "'"
        if group_str != "":
            qry_where += " AND work_group = '" + group_str + "'"
        if school_str != "":
            qry_where += " AND school = '" + school_str + "'"
        if ctry_str != "":
            qry_where += " AND country = '" + ctry_str + "'"
        print(qry_where)
        result = db_conn.get_values("affiliations","id",qry_where)
        if result != []:
            affi_id = result[0][0]
            print("Found:", affi_id, len(result))
            add_id = db_conn.get_value("addresses", "id", "affiliation_id", affi_id)[0]
            print('affiliation id:', affi_id, " address id: ", add_id)
            return affi_id, add_id
        # not found, add new institution record
        else:
            affi_id, add_id = add_affi_to_db(inst_str, dept_str, faculty_str, group_str, ctry_str, school_str, addr_list, qry_where)
            return affi_id, add_id
    return 0, 0

def extract_custom(split_this):
    print( split_this)
    decimal_str = ""
    for indx in range(0, len(split_this)):
        val = " "
        if (indx % 10 == 0): val = str(int(indx/10)) 
        decimal_str += val
    print(decimal_str)
    unit_str = ""
    for indx in range(0, len(split_this)):
        val = str(indx%10)
        if (indx % 10 == 0): val = "0" 
        unit_str += val
    print(unit_str)
    print("start")
    str_start = int(input())
    print("end")
    str_end = int(input())
    return split_this[str_start:str_end+1]

def is_address(affi_element):
    while True:
        print ("************************SELECT OPTION*************************")
        print (affi_element,": \n a) address \n n) new affi \n s) str affi")
        print ("selection:", end=" ")
        user_select = input().lower()
        if user_select == 'a':
            return 1
        elif user_select == 'n':
            return 0
        elif user_select == 's':
            return -1

def assing_keyword(affi_element):
    while True:
        print ("****************ASSIGN KEYWORD TO ELEMENT****************")
        print ("* Element:",affi_element)
        for opt in affi_keys:
            print (opt+")", affi_keys[opt])
        print ("x) None")
        print ("selection:", end=" ")
        user_select = input().lower()
        if user_select in affi_keys:
            return affi_keys[user_select]
        elif user_select == "x":
            return ""

def assign_not_mapped(list_affis, not_mapped):
    for element in not_mapped:
        print(element)
        print ("************************MAPPED AFFIS*************************")
        affi_keyword = assing_keyword(element)
        for i, affi in enumerate(list_affis):
            print (i, affi)
        while True:
            print ("******************ASSIGN NOT MAPPED TO AFFIS******************")
            print(element)
            print ("select number to assing to from above or 9 to assign to new affiliation")
            print ("selection:", end=" ")
            user_select = input()
            try:
                user_select = int(user_select)
                if user_select < len(list_affis):
                    if not affi_keyword in list_affis[user_select].keys():
                        list_affis[user_select][affi_keyword] = element
                        break;
                    else:
                        print ("cannot add")
                elif user_select == 9:
                    list_affis.append({affi_keyword:element})
                    break
            except:
                print("select a valid index")
        return list_affis
    
def get_keyword(affi_element):
    if affi_element in institutions_list:
        return "institution"
    elif affi_element in institution_synonyms:
        return "institution"
    elif affi_element in department_list:
        return "department"
    elif affi_element in faculty_list:
        return "faculty"
    elif affi_element in group_list: 
        return "work_group"
    elif affi_element in countries_list:
        return "country"
    elif affi_element in country_synonyms:
        return "country"
    else:
        return ""

# returns a list of dictionaries with affiliation values
def map_split_affi(affiliation):
    parsed_affi = { }
    affis_count = 0
    list_affis = []
    addr_list = []
    not_mapped = []
    for affi_element in affiliation:
        element_value = affi_element['name']
        element_key = get_keyword(element_value)
        if element_key == "":
            # Not found it could be a single string or an address element
            # ask if address or new affi
            is_addr = is_address(element_value)
            if is_addr == 1: # address part
                addr_list.append(element_value)
            elif is_addr == 0: # new affi part
                not_mapped.append(element_value)
            elif is_addr == -1: # new affi part
                print("not handled", element_value)
        else:
            if not element_key in parsed_affi:
                parsed_affi[element_key] = element_value
            else:
                # There is more than one affiliation
                parsed_affi['addr'] = addr_list
                list_affis.append(parsed_affi)
                parsed_affi = {}
                addr_list = []
                parsed_affi[element_key] = element_value
                affis_count += 1
        if affis_count > 0 and parsed_affi != {}:
            if addr_list != []:
                parsed_affi['addr'] = addr_list
            list_affis.append(parsed_affi)
    return list_affis, not_mapped

def get_afi_id(affiliation):
    print (affiliation, len(affiliation))
    inst_str = dept_str = faculty_str = group_str = ctry_str = ""
    qry_where_str = ""
    addr_list=[]
    for indx in range(0, len(affiliation)):
        checking_this = affiliation[indx]['name']
        #print (affiliation[indx]['name'])
        if checking_this in institutions_list:
            if inst_str == "":
                inst_str = checking_this
        elif checking_this in department_list:
            if inst_str == "":
                dept_str = checking_this
        elif checking_this in faculty_list:
            faculty_str = checking_this
        elif checking_this in group_list: 
            group_str = checking_this
        elif checking_this in countries_list:
            ctry_str = checking_this
        elif checking_this in country_synonyms:
            ctry_str = country_synonyms[checking_this]
        elif checking_this in institution_synonyms:
            inst_str = institution_synonyms[checking_this]
        else:
            addr_list.append(checking_this)
            
    qry_where_str += "institution = '" + inst_str + "'"        
    qry_where_str += " AND department = '" + dept_str + "'"
    qry_where_str += " AND faculty = '" + faculty_str + "'"
    qry_where_str += " AND work_group = '" + group_str + "'"
    qry_where_str += " AND country = '" + ctry_str + "'"

    print ('Institution:', inst_str)
    print ('Department:', dept_str)
    print ('Faculty:', faculty_str)
    print ('Group:', group_str)
    print ('Country:', ctry_str) 
    print ('Address',  addr_list)
    print (qry_where_str)
    result = db_conn.get_values("affiliations","id",qry_where_str)
    print("found",result)
    if result != []:
            affi_id = result[0][0]
            print("Found:", affi_id, len(result))
            add_id = db_conn.get_value("addresses", "id", "affiliation_id", affi_id)[0]
            print('affiliation id:', affi_id, " address id: ", add_id)
            addr_row = list(db_conn.get_row("addresses", add_id)[0])
            print(addr_row)
            another_affi=[]
            for indx in range(0, len(affiliation)):
                checking_this = affiliation[indx]['name']
                print(checking_this)
                if not checking_this in [inst_str, dept_str, faculty_str, group_str, ctry_str]:
                    if not(checking_this in country_synonyms.keys()):
                        skip = False
                        for add_itm in addr_row:
                            if str(add_itm) in checking_this:
                                skip = True
                                break;
                        if not skip:
                            another_affi.append({'name':checking_this})
                        
            if another_affi != []:
                get_afi_id(another_affi)
            return affi_id, add_id
    else:
        # add new affiliation
        affi_id, add_id = add_affi_to_db(inst_str, dept_str, faculty_str, group_str, ctry_str, addr_list,qry_where_str)
        return affi_id, add_id


In [ ]:
def copy_author_affi(art_aut_id, affi_id):
    new_id = None
    values_list = db_conn.get_values("author_affiliations", "id", "affiliation_id = "+str(affi_id))
    
    row_values = []
    if len(values_list)>0:
        row_values = db_conn.get_row ("author_affiliations",values_list[0][0])

    if len(row_values) > 0: 
        # get the first element, ignoring the ID 
        row_values = list(row_values[0][1:])
        #replace art_aut_id 
        row_values[0] = art_aut_id
        row_values[10] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        row_values[11] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        row_keys = ['article_author_id','name', 'short_name', 'add_01', 
                    'add_02', 'add_03','add_04','add_05','country',
                    'affiliation_id','created_at','updated_at']
        new_id = db_conn.put_values_table("author_affiliations", row_keys, row_values)
    return new_id

In [ ]:
aut_affi_ids = db_conn.get_values('cr_affiliations', 'article_author_id',
                                  'author_affiliation_id is NULL GROUP BY article_author_id HAVING count() = 1')
flat_list = []
for sublist in aut_affi_ids:
    for item in sublist:
        flat_list.append(item)

i_idx = 0
aut_affi_ids = list(set(flat_list))
for aai in aut_affi_ids:
    print("Art. author", aai)
    non_parsed_affis = db_conn.get_values('cr_affiliations', 'name', 'article_author_id = '+ str(aai))
    non_parsed_cr_affi_ids = db_conn.get_values('cr_affiliations', 'id', 'article_author_id = '+ str(aai))
    json_affis = []
    for sublist in non_parsed_affis:
        for item in sublist:
            json_affis.append({'name':item})
            
    print("PARSING", json_affis)
    list_affis, not_mapped = map_split_affi(json_affis)
    if not_mapped != []:
        print('****** There are some not mapped affis ******')
        print('* Not mapped: ', not_mapped)
        list_affis = assign_not_mapped(list_affis, not_mapped)
    print('* List mapped affis:', list_affis)
    print('*********************************************')
    if len(json_affis) == 1:
        affiliation_id, address_id = db_split(json_affis)
        print("Returned IDs:", affiliation_id, address_id, "for art. author", aai)
        # add author affiliation record
        aut_affi_id = copy_author_affi(aai, affiliation_id)
        print ("Inserted author affiliation:", aut_affi_id )
        #if there is only one then there is no problem for adding 
        if len(non_parsed_cr_affi_ids) == 1:
            db_conn.set_value_table('cr_affiliations', non_parsed_cr_affi_ids[0][0], 'author_affiliation_id', aut_affi_id)
            print("Updated CR affiliation:", non_parsed_cr_affi_ids[0][0])
    elif len(json_affis) > 1:
        break
        affiliation_id, address_id = db_split(json_affis)
        print("Returned IDs:", affiliation_id, address_id, "for art. author", aai)
    elif len(json_affis) > 5:
        get_afi_id(json_affis)
    if i_idx == 9:
        break
    i_idx+=1

In [ ]:
# test str_has_synonym
test_str = "Beamline I22, Diamond Light Source, Harwell Science and Innovation Campus, Didcot OX11 0DE, U.K."
inst_str, parsed_str = str_has_synonym(test_str, institution_synonyms)

if inst_str != "Diamond Light Source Ltd.": 
    print ("str_has_synonym failed, returned\n\t", inst_str, "expected\n\t", "Diamond Light Source Ltd.")    
elif parsed_str != "Beamline I22, , Harwell Science and Innovation Campus, Didcot OX11 0DE, U.K.":
    print ("str_has_synonym failed, returned\n\t", parsed_str,"expected\n\t","Beamline I22, , Harwell Science and Innovation Campus, Didcot OX11 0DE, U.K." )
else:
    print ("str_has_synonym works")
    

test_str = "Beamline I22, Diamond Light Source, Harwell Science and Innovation Campus, Didcot OX11 0DE, U.K."
inst_str, parsed_str = str_has_synonym(test_str, country_synonyms)

if inst_str != "United Kingdom": 
    print ("str_has_synonym failed, returned\n\t", inst_str, "expected\n\t", "United Kingdom")    
elif parsed_str != "Beamline I22, Diamond Light Source, Harwell Science and Innovation Campus, Didcot OX11 0DE, ":
    print ("str_has_synonym failed, returned\n\t", parsed_str,"expected\n\t","Beamline I22, Diamond Light Source, Harwell Science and Innovation Campus, Didcot OX11 0DE, " )
else:
    print ("str_has_synonym works")


In [ ]:
aut_affi = {'institution': '', 'school': '', 'department': '', 'faculty': '', 'group': 'Chemical Research Laboratory', 'country': 'United Kingdom', 'address': 'Oxford Chemistry   12 Mansfield Road Oxford OX1 3TA'}
print(aut_affi)
del aut_affi['address']
print(aut_affi)
del aut_affi['address']
print(aut_affi)

In [ ]:
non_parsed_affi_ids = db_conn.get_values('cr_affiliations', 'id', 'article_author_id = '+ str(aai))

In [ ]:
non_parsed_affi_ids[0][0]

In [ ]:
str_this = ', , Nakhon Pathom, , , '
str_this = str_this.replace(", ,", "")
str_this = str_this.replace(" ,", ",")
str_this = str_this.strip()
if str_this[0] =="," : str_this = str_this[1:] 
if str_this[-1:] =="," : str_this = str_this[:-1] 
str_this
#str_this[0] =="," then

In [ ]:
str_this[:-1]

In [ ]:
affiliation_values = {'institution': '', 'school': 'School of Chemistry and Chemical Engineering', 
                      'department': '',  'faculty': '', 'work_group': '', 'country': 'United Kingdom', 
                       'address': 'Queen’s University of Belfast, University Road, Belfast BT9 5AG, Northern Ireland'}
affiliation_values = {'institution': '', 'school': '', 'department': '', 'faculty': '', 'work_group': '',
                      'country': 'United Kingdom',
                      'address': 'School of Biological and Chemical SciencesMaterials Research InstituteQueen Mary University of London London E1 4NS'}

In [ ]:
opt_end = False
while (opt_end == False):
    print(affiliation_values)
    missing_vals = {}
    int_idx = 1
    for an_element in affiliation_values:
        if affiliation_values[an_element] == "":
            missing_vals[int_idx] = an_element
            int_idx += 1
    print('Which element to get from string:')
    for an_idx in missing_vals:
        print(" ", an_idx, missing_vals[an_idx])
    print (" ", len(missing_vals)+1, 'end')
    print('Selection')
    option_sel = int(input())
    if option_sel == len(missing_vals)+1:
        opt_end = True
    elif option_sel > 0 and option_sel <= len(missing_vals):
        print("Try to extract ", missing_vals[option_sel])
        affiliation_values[missing_vals[option_sel]], affiliation_values['address'] = get_from_string(affiliation_values['address'], missing_vals[option_sel])

In [ ]:
institutions_list

In [ ]:
splitting_this ="School of Chemistry and Chemical Engineering, Queen’s University Belfast, David-Keir Building, Stranmillis Road, Belfast BT9 5AG, U.K."
inst_str, splitting_this = check_list(splitting_this,institutions_list)
inst_str

In [ ]:
splitting_this

In [ ]:
return_str = temp_str = ""
for an_item in institutions_list:
    if "Queen" in an_item:
        print(an_item)
    if an_item in splitting_this:
        if len(an_item) > len(temp_str):
            temp_str = an_item
if len(temp_str)>0:
    splitting_this = splitting_this.replace(temp_str,"")
    return_str = temp_str
print(return_str, splitting_this)

In [ ]:
"Queen's University Belfast" in "School of Chemistry and Chemical Engineering, Queen’s University Belfast, David-Keir Building, Stranmillis Road, Belfast BT9 5AG, U.K."

In [ ]:
len(aut_affi_ids)
len(non_parsed_affis)
len(non_parsed_cr_affi_ids)

In [ ]:
print(aut_affi_id)
print(non_parsed_affis)
print(non_parsed_cr_affi_ids)

In [ ]:
aut_affis= [('Johnson Matthey Technology Centre, Sonning Common, Reading, RG4 9NH United Kingdom',)]
len(aut_affis[0])